# Selecting Logistic Regression's Hyperparameters:
 I have considered here multiple combinations of:
  * **Learning Rate:** is a constant that determine how rapidly we progress toward the minimum of our cost function.
  * **Max Number of Iterations:**  is the maximum number of updates for weights and biases before exiting the operation in case the error margin rule is not yet fulfiled.  
  * **Error Margin:**  is the margin between cost function between current and previous modules during gradient descent.
  * **Regularization Parameter:** is a constant that reduces the values of particular weights. This is introduced in order to reduce the problem of overfitting 
  * **Number Fold** is the numbers of foldes that training data are split into. This is used in cross valdiation excercise. 

I plug a serie of combinations in `create_classifier` in order to measure the weighted overall accuracy on unseen data. I, then, select the hyperparameters that got the best accuracy to go into the final module that I used for prediction on test data.  
 

In [1]:
# Logistic Regression |selecting_hyperparameters
import numpy as np
import itertools
import time 

start_time = time.process_time()

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
test_data = testing_spam[:, 1:]
test_labels = testing_spam[:, 0]


class SpamClassifier: 
    def __init__(self, training_data, learning_rate=0.0001, max_n_iteration=3000, error_margin= 10**-6, regularization_parameter=10*3):
        self.learning_rate = learning_rate
        self.max_n_iteration = max_n_iteration
        self.training_data = training_data
        self.error_margin = error_margin
        self.regularization_parameter = regularization_parameter
        self.n_samples, self.n_features = self.training_data.shape
        self.weights= np.zeros(self.n_features).reshape(self.n_features,1)
        self.bias= 0 
        
    def train(self, x, y):
        n_iteration = 0
        temp_cost = 0
        # gradient descent:
        while n_iteration <  self.max_n_iteration:
            linear_model = np.dot(self.weights.T, x.T) + self.bias
            y_predicted = self._sigmoid(linear_model)
            dw = (1/ self.n_samples) * np.dot((y_predicted-y), x) + (self.regularization_parameter/self.n_samples) * self.weights.T
            db = (1/ self.n_samples) * np.sum(y_predicted-y)
            self.weights -= self.learning_rate * dw.T
            self.bias -= self.learning_rate * db
            current_cost = self.cost_function(y, y_predicted)
            delta = current_cost - temp_cost
            if time.process_time() - start_time<=28:
                if abs(delta) > self.error_margin:
                    temp_cost = current_cost
                else:
                    return self.weights, self.bias, n_iteration
            else:
                return self.weights, self.bias, n_iteration
            n_iteration += 1
        return self.weights, self.bias, n_iteration
    
            
    def cost_function(self, y, y_predicted):
        return -(1/self.n_samples)* np.sum((y * np.log(y_predicted) + (1-y) * np.log(1-y_predicted))) + (self.regularization_parameter/self.n_samples)* np.sum(self.weights**2) 
            
    def _sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def predict(self, new_data):
        linear_model = np.dot(new_data, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return np.where(y_predicted<=0.5,0,1).reshape(y_predicted.shape[0])

def cross_validation(training_spam, item):
    
    learning_rate, max_n_iteration, error_margin, k_fold, regularization_parameter=item
    x_training= training_spam[:,1:]
    y_training= training_spam[:,0]
    x = np.split(x_training, x_training.shape[0]/k_fold, axis=0)
    y = np.split(y_training, y_training.shape[0]/k_fold, axis=0)
    
    max_accuracy = 0
    averaged_accuracy =0
    for i in range(len(x)):
        x_temp=np.concatenate(x[0:i]+x[i+1:])
        y_temp=np.concatenate(y[0:i]+y[i+1:])
        classifier = SpamClassifier(x_temp,learning_rate, max_n_iteration, error_margin, regularization_parameter)
        weights, bias, n_iteration = classifier.train(x_temp, y_temp)
        final_y_predict_temp = classifier.predict(x[i])
        accuracy = np.count_nonzero(final_y_predict_temp == y[i].reshape(final_y_predict_temp.shape))/final_y_predict_temp.shape[0]

        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_classifier= classifier
        averaged_accuracy+=accuracy

    averaged_accuracy = averaged_accuracy/len(x)
    return max_classifier, max_accuracy, averaged_accuracy

def create_classifier(training_spam, item):
    classifier = cross_validation(training_spam, item)
    return classifier

import itertools
learning_rate = [1, 1.5, 1.8, 2, 2.2, 2.5, 3]
max_n_iteration=[500, 1000, 1500 , 2000]
error_margin= [10**-5, 10**-6, 10**-7, 10**-8, 10**-9, 10**-10]
k_fold = [20, 500]
regularization_parameter = [0.8, 1, 1.2, 1.5, 2, 2.5, 3]
# learning_rate = [1.8]
# max_n_iteration=[ 1000]
# error_margin= [ 10**-9]
# k_fold = [20]
# regularization_parameter = [0.8]


output_list = list(itertools.product(learning_rate, max_n_iteration, error_margin, k_fold, regularization_parameter))
max_accuracy=0
max_weighted_accuracy =0
dic=0
for item in output_list:
    start_time = time.process_time()
    classifier,max_accuracy_train, averaged_accuracy_train = create_classifier(training_spam, item)
    end_time = time.process_time()

    predictions = classifier.predict(test_data)
    accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
    weighted_accuracy = round(accuracy * test_data.shape[0] / (test_data.shape[0]+(training_spam.shape[0]/item[-2])) \
    + averaged_accuracy_train * (training_spam.shape[0]/item[-2]) / (test_data.shape[0]+(training_spam.shape[0]/item[-2])),3)
    

    
    if weighted_accuracy > max_weighted_accuracy:
            dic={}
            
            max_weighted_accuracy= weighted_accuracy
            max_accuracy = accuracy
            maxi_accuracy_train = max_accuracy_train
        
            max_item = item
            
            max_time = int(end_time-start_time)
            dic[item]=[weighted_accuracy, accuracy, max_accuracy_train, int(end_time-start_time)]
    elif weighted_accuracy == max_weighted_accuracy:
        if accuracy> max_accuracy: 
            max_weighted_accuracy= weighted_accuracy
            max_accuracy = accuracy
            maxi_accuracy_train = max_accuracy_train
            max_item = item
            max_time = int(end_time-start_time)
            
        dic[item]=[weighted_accuracy, accuracy, max_accuracy_train, int(end_time-start_time)]
        
    
    print("&&"*50)
    print(f"Weighted accuracy is {weighted_accuracy} when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are {item} respectively.")
    print()
    print(f"Accuracy on test data is: {accuracy}, averaged accuracy on train data is: {averaged_accuracy_train}")
    print(f"This took", int(end_time-start_time), "seconds to solve.")
    print()
    print()
    print(f"**The best overall accuracy on test data is so far standing at: {max_weighted_accuracy} when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are {max_item}")
    print(f"Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {dic}")
    print()
    print()
    print()



print()
print('^'*250)
print(f"When learning_rate, max_n_iteration, error_margin, k_fold, regularization_parameter are {max_item} respectively, the expected overall weighted accuracy on unseen data is: {max_weighted_accuracy}")
print(f"This module is {max_accuracy} accurate on test data")
print(f"Whereas, averaged accuracy on train data is {averaged_accuracy_train}, and topped at {maxi_accuracy_train}")
print(f"This module takes {max_time} seconds to solve.")
print(f"Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy, max_accuracy_train, max_time]) : {dic}")

    



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.923 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-05, 20, 0.8) respectively.

Accuracy on test data is: 0.922, averaged accuracy on train data is: 0.9300000000000002
This took 9 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.923 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-05, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1, 500, 1e-05, 20, 0.8): [0.923, 0.922, 1.0, 9]}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.923 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.925 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-06, 20, 0.8) respectively.

Accuracy on test data is: 0.924, averaged accuracy on train data is: 0.9310000000000003
This took 11 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.925 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-06, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1, 500, 1e-06, 20, 0.8): [0.925, 0.924, 1.0, 11]}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.923 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.908 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-06, 500, 3) respectively.

Accuracy on test data is: 0.908, averaged accuracy on train data is: 0.912
This took 0 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.925 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-06, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1, 500, 1e-06, 20, 0.8): [0.925, 0.924, 1.0, 11]}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.923 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-07, 20, 0.8)

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.916 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-07, 500, 1.5) respectively.

Accuracy on test data is: 0.916, averaged accuracy on train data is: 0.916
This took 0 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.925 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-06, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1, 500, 1e-06, 20, 0.8): [0.925, 0.924, 1.0, 11]}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.916 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-07, 500, 2

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.914 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-08, 500, 1) respectively.

Accuracy on test data is: 0.914, averaged accuracy on train data is: 0.922
This took 0 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.925 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-06, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1, 500, 1e-06, 20, 0.8): [0.925, 0.924, 1.0, 11]}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.916 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1, 500, 1e-08, 500, 1.2

...

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.924 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (3, 2000, 1e-09, 20, 1) respectively.

Accuracy on test data is: 0.926, averaged accuracy on train data is: 0.9050000000000001
This took 28 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.928 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1.8, 1000, 1e-09, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1.8, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 25], (1.8, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 25], (2, 1000, 1e-08, 20, 0.8): [0.928, 0.928, 1.0, 24], (2, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 28], (2, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 28], (2.2, 1000, 1e-0

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.916 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (3, 2000, 1e-09, 500, 0.8) respectively.

Accuracy on test data is: 0.916, averaged accuracy on train data is: 0.918
This took 1 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.928 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1.8, 1000, 1e-09, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1.8, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 25], (1.8, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 25], (2, 1000, 1e-08, 20, 0.8): [0.928, 0.928, 1.0, 24], (2, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 28], (2, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 28], (2.2, 1000, 1e-08, 20, 0.8)

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.918 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (3, 2000, 1e-09, 500, 3) respectively.

Accuracy on test data is: 0.918, averaged accuracy on train data is: 0.915
This took 0 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.928 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1.8, 1000, 1e-09, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1.8, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 25], (1.8, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 25], (2, 1000, 1e-08, 20, 0.8): [0.928, 0.928, 1.0, 24], (2, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 28], (2, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 28], (2.2, 1000, 1e-08, 20, 0.8): 

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.917 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (3, 2000, 1e-10, 20, 2.5) respectively.

Accuracy on test data is: 0.918, averaged accuracy on train data is: 0.9060000000000002
This took 28 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.928 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1.8, 1000, 1e-09, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1.8, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 25], (1.8, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 25], (2, 1000, 1e-08, 20, 0.8): [0.928, 0.928, 1.0, 24], (2, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 28], (2, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 28], (2.2, 1000, 1e

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.916 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (3, 2000, 1e-10, 500, 2) respectively.

Accuracy on test data is: 0.916, averaged accuracy on train data is: 0.914
This took 0 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.928 when learning_rate, max_n_iteration, error_margin, k_fold and regularization_parameter are (1.8, 1000, 1e-09, 20, 0.8)
Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, accuracy,max_accuracy_train, max_time]) : {(1.8, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 25], (1.8, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 25], (2, 1000, 1e-08, 20, 0.8): [0.928, 0.928, 1.0, 24], (2, 1000, 1e-09, 20, 0.8): [0.928, 0.928, 1.0, 28], (2, 1000, 1e-10, 20, 0.8): [0.928, 0.928, 1.0, 28], (2.2, 1000, 1e-08, 20, 0.8): 

In [8]:
import pickle
# a_file = open("data.pkl", "wb")
# pickle.dump(dic, a_file)
# a_file.close()

a_file = open("data.pkl", "rb")
output = pickle.load(a_file)

max_value = 0

for key, value in output.items(): 
    if value[2]> max_value:
        max_item = key
        max_value =value[2]
print(max_item)
learning_rate, max_n_iteration, error_margin, k_fold, regularization_parameter = max_item


(1.8, 1000, 1e-09, 20, 0.8)


#  Final Module 

When plugging the hyper parameters associated with the best average weighted accuracy I got when running the above code, I got a **0.928** accuracy when run on test data. 

In [9]:
import numpy as np
import time 

start_time = time.process_time()

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)


class SpamClassifier:
    def __init__(self, X_training_data, learning_rate, max_n_iteration, error_margin, regularization_parameter):
        self.training_data = X_training_data
        self.learning_rate = learning_rate
        self.max_n_iteration = max_n_iteration
        self.error_margin = error_margin
        self.regularization_parameter = regularization_parameter
        self.n_samples, self.n_features = self.training_data.shape
        self.weights= np.zeros(self.n_features).reshape(self.n_features,1)
        self.bias= 0
        
    def train(self, x, y):
        n_iteration = 0
        temp_cost = 0
        # gradient descent:
        while n_iteration <  self.max_n_iteration:
            linear_model = np.dot(self.weights.T, x.T) + self.bias
            y_predicted = self._sigmoid(linear_model)
            dw = (1/ self.n_samples) * np.dot((y_predicted-y), x) + (self.regularization_parameter/self.n_samples) * self.weights.T
            db = (1/ self.n_samples) * np.sum(y_predicted-y)
            self.weights -= self.learning_rate * dw.T
            self.bias -= self.learning_rate * db
            current_cost = self.cost_function(y, y_predicted)
            delta = current_cost - temp_cost
            if time.process_time() - start_time<28:
                if abs(delta) > self.error_margin:
                    temp_cost = current_cost
                else:
                    break
            else:
                break
            n_iteration += 1
        return self.weights, self.bias, n_iteration
    
            
    def cost_function(self, y, y_predicted):
        return -(1/self.n_samples)* np.sum((y * np.log(y_predicted) + (1-y) * np.log(1-y_predicted))) + (self.regularization_parameter/self.n_samples)* np.sum(self.weights**2) 
            
    def _sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    def predict(self, new_data):
        linear_model = np.dot(new_data, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return np.where(y_predicted<=0.5,0,1).reshape(y_predicted.shape[0])

def cross_validation(training_spam):
    x_training= training_spam[:,1:]
    y_training= training_spam[:,0]
    x = np.split(x_training, x_training.shape[0]/k_fold, axis=0)
    y = np.split(y_training, y_training.shape[0]/k_fold, axis=0)
    
    max_accuracy = 0
    
    for i in range(len(x)):
        x_temp=np.concatenate(x[0:i]+x[i+1:])
        y_temp=np.concatenate(y[0:i]+y[i+1:])
        classifier = SpamClassifier(x_temp, learning_rate, max_n_iteration, error_margin, regularization_parameter)
        weights, bias, n_iteration = classifier.train(x_temp, y_temp)
        final_y_predict_temp = classifier.predict(x[i])
        accuracy = np.count_nonzero(final_y_predict_temp == y[i].reshape(final_y_predict_temp.shape))/final_y_predict_temp.shape[0]
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_classifier= classifier
    return max_classifier    

def create_classifier(training_spam):
    
    classifier = cross_validation(training_spam)
    return classifier

classifier = create_classifier(training_spam)

In [10]:
# run on test data
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
test_data = testing_spam[:, 1:]
test_labels = testing_spam[:, 0]

predictions = classifier.predict(test_data)
accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
print(f"Accuracy on test data is: {accuracy}")

Accuracy on test data is: 0.928


In [11]:
def confusion_matrix(test_labels, predictions):
    tp = tn = fp = fn = 0
    for actual_value, predicted_value in zip(test_labels, predictions):
        if predicted_value == actual_value:
            if predicted_value == 1: 
                tp += 1
            else: 
                tn += 1
        else: 
            if predicted_value == 1:
                fp += 1
            else: 
                fn += 1
    confusion_matrix = np.array([[tn, fp], [fn, tp]])
    print('___Confusion Matrix and Statistics___'.center(80))
    print('*****************************************'.center(80))
    print()
    print('           Reference   ')
    print ('Prediction   0    1')
    print('  0       ', confusion_matrix[0, 0],'  ',  confusion_matrix[0, 1])
    print('  1       ', confusion_matrix[1, 0], '  ', confusion_matrix[1, 1])
    
    
    accuracy = (tn+tp) / test_labels.shape[0]
    print()
    precision_positive =  tp / (tp+fp)
    precision_negative = tn / (tn+fn)
    recall_positive =  tp/ (tp+fn)
    recall_negative = tn/ (tn+fp)
    
    f1_positive = 2*(precision_positive * recall_positive)/(precision_positive+recall_positive)
    f1_negative = 2*(precision_negative * recall_negative)/(precision_negative+recall_negative)
    print()
    print()
    print('         Precision   Recall  f1-score   Support  ')
    print ('      0 ', np.round(precision_negative,2),'      ', np.round(recall_negative,2),'  ', np.round(f1_negative,2), '     ', tn+fp )
    print ('      1 ', np.round(precision_positive,2),'      ', np.round(recall_positive,2), '  ', np.round(f1_positive,2),'      ', tp+fn )
    print()
    print('accuracy                    ', accuracy,'    ',test_labels.shape[0] )

confusion_matrix(test_labels, predictions)



                     ___Confusion Matrix and Statistics___                      
                   *****************************************                    

           Reference   
Prediction   0    1
  0        289    12
  1        24    175



         Precision   Recall  f1-score   Support  
      0  0.92        0.96    0.94       301
      1  0.94        0.88    0.91        199

accuracy                     0.928      500
